In [1]:
import os
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import StorageLevel, SparkConf, SparkContext
import sparknlp

print(_find_spark_home())

/home/zzfz/anaconda3/envs/BigData/lib/python3.10/site-packages/pyspark


In [3]:
# conf = (
#     SparkConf()
#     .setMaster("local[*]")
#     .set("spark.local.dir", "/home/zzfz/spark-temp")
#     .set("spark.driver.memory", "6g")
#     .set("spark.executor.memory", "12g")
#     .set("spark.driver.maxResultSize", "10g")
#     .set("spark.pyspark.python", "/home/zzfz/anaconda3/envs/BigData/bin/python")
#     .set("spark.pyspark.driver.python", "/home/zzfz/anaconda3/envs/BigData/bin/python")
# )

# spark = (
#     SparkSession.builder.appName("PySpark Hadoop Integration")
#     .config(conf=conf)
#     .getOrCreate()
# )

spark = sparknlp.start()

sc = spark.sparkContext

for item in sc.getConf().getAll():
    print(item)

your 131072x1 screen size is bogus. expect trouble
23/12/23 15:29:19 WARN Utils: Your hostname, GinXPS15 resolves to a loopback address: 127.0.1.1; using 172.29.117.66 instead (on interface eth0)
23/12/23 15:29:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zzfz/anaconda3/envs/BigData/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zzfz/.ivy2/cache
The jars for the packages stored in: /home/zzfz/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3f66f4f8-0d25-45bb-b6f8-695e5fdf6f03;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureacc

('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.driver.memory', '16G')
('spark.app.initial.file.urls', 'file:///home/zzfz/.ivy2/jars/javax.annotati

In [5]:
df = (
    spark.read.option("delimiter", "\t")
    .option("header", "true")
    .csv("hdfs://localhost:9000/datasets/Amazon_Reviews/*.tsv")
)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [6]:
sampled_df = df.sample(fraction=0.00001, seed=3)

sampled_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47294965|R1N7I7QI765GI3|B00WAJE1IG|     685353023|What Happens On T...|         Apparel|          5|            0|          0|   N|                Y|          Five Stars|Kind of says it a...| 2015-07-22|
|         US|   24113674| R7IYW0BF9CISR|B00VISB8MM|     695333856|Men's Teens 6 Pai...|         Apparel|          4|    

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
from pyspark.sql.functions import col, count, when

# Calculate the total number of rows
total_rows = df.count()

# Calculate the ratio of nulls for each column
null_ratios = df.select(
    [((count(when(col(c).isNull(), c))) / total_rows).alias(c) for c in df.columns]
)

# Show the null ratios
null_ratios.show()

+-----------+-----------+---------+----------+--------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|    product_category|         star_rating|       helpful_votes|         total_votes|                vine|   verified_purchase|     review_headline|         review_body|         review_date|
+-----------+-----------+---------+----------+--------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        0.0|        0.0|      0.0|       0.0|           0.0|          0.0|1.596095511520841...|1.627052298395746...|1.633425754517050...|1.633425754517050...|1.633425754517050...|1.633425754517050...|1.86

In [19]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, UniversalSentenceEncoder, SentimentDLModel
from pyspark.ml import Pipeline


def create_sentiment_pipeline(input_col):
    # Assemble Document
    documentAssembler = (
        DocumentAssembler().setInputCol(input_col).setOutputCol("document")
    )

    # Tokenize the document
    tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

    # Use Universal Sentence Encoder for embeddings
    embeddings = (
        UniversalSentenceEncoder.pretrained()
        .setInputCols(["document"])
        .setOutputCol("sentence_embeddings")
    )

    # Load the pre-trained sentiment analysis model
    sentimentModel = (
        SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")
        .setInputCols(["sentence_embeddings"])
        .setOutputCol("sentiment")
    )

    # Define the pipeline
    pipeline = Pipeline(
        stages=[documentAssembler, tokenizer, embeddings, sentimentModel]
    )

    return pipeline


# Create pipelines for review_headline and review_body
pipeline_headline = create_sentiment_pipeline("review_headline")
pipeline_body = create_sentiment_pipeline("review_body")

# Fit and transform the pipelines
pipeline_model_headline = pipeline_headline.fit(df)
result_headline = pipeline_model_headline.transform(df)

pipeline_model_body = pipeline_body.fit(df)
result_body = pipeline_model_body.transform(df)

# Display results
result_headline.select("review_headline", "sentiment.result").show(truncate=False)
result_body.select("review_body", "sentiment.result").show(truncate=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]
+-------------------------------------------------------+----------+
|review_headline                                        |result    |
+-------------------------------------------------------+----------+
|★ THESE REALLY DO WORK GREAT WITH SOME TWEAKING ★      |[positive]|
|Favorite for winter. Very warm!                        |[positive]|
|Great Socks for the money.                             |[positive]|
|Slick hat!                                             |[positive]|
|I would do it again!                                   |[positive]|
|Five Stars                           